# Разметка данных

# Пакеты

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from enum import Enum
from dataclasses import dataclass, field
from datasets import Dataset, DatasetDict
%matplotlib inline

In [60]:
df = pd.read_csv("gar_places.csv")
df = df.drop(['oktmo', 'oktmo_row_number', 'mun_area_name', 'mun_area_type_name', 'mun_area_type_name_short', 'settlement_name', 'settlement_type_name', 'settlement_type_name_short'], axis=1)
df = df.fillna('')

In [61]:
df

,region_name,region_type_name,region_type_name_short,area_name,area_type_name,area_type_name_short,city_name,city_type_name,city_type_name_short,territory_name,territory_type_name,territory_type_name_short,street_name,street_type_name,street_type_name_short
0,Алтайский,Край,край,Залесовский,Район,р-н,Залесово,Село,с,,,,Полевая,Улица,ул
1,Алтайский,Край,край,Залесовский,Район,р-н,Залесово,Село,с,автодорога Мартыново-Тогул-Залесово,Территория,тер.,151-й,Километр,км
2,Алтайский,Край,край,Залесовский,Район,р-н,Залесово,Село,с,,,,Ворошилова,Улица,ул
3,Алтайский,Край,край,Залесовский,Район,р-н,Залесово,Село,с,,,,Красноармейский,Переулок,пер
4,Алтайский,Край,край,Залесовский,Район,р-н,Залесово,Село,с,,,,Гагарина,Улица,ул
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612825,Еврейская,Автономная область,Аобл,,,,Биробиджан,Город,г,СОТ Биршоссе 13 км,Территория,тер.,Щукинка,Улица,ул
612826,Еврейская,Автономная область,Аобл,,,,Биробиджан,Город,г,СОТ Биршоссе 13 км,Территория,тер.,Западная,Улица,ул
612827,Еврейская,Автономная область,Аобл,,,,Биробиджан,Город,г,СОТ Биршоссе 13 км,Территория,тер.,Поворотная,Улица,ул
612828,Еврейская,Автономная область,Аобл,,,,Биробиджан,Город,г,СОТ Багульник,Территория,тер,Четвертая,Улица,ул


# Словарь Entities

In [62]:

class ElementType(Enum):
    REGION=0
    AREA=2
    TERRITORY=4
    CITY=6
    STREET=8

    def begin_tag_name(cls):
        return 'B-'+cls.name
    def inside_tag_name(cls):
        return 'I-'+cls.name
    def begin_tag_code(cls):
        return cls.value.real+0
    def inside_tag_code(cls):
            return cls.value.real+1


In [63]:
print(ElementType.REGION.begin_tag_name())

B-REGION


In [64]:
all_tags = []
for type in ElementType:
    all_tags = all_tags + [type.begin_tag_name(), type.inside_tag_name()]
print(all_tags)   

['B-REGION', 'I-REGION', 'B-AREA', 'I-AREA', 'B-TERRITORY', 'I-TERRITORY', 'B-CITY', 'I-CITY', 'B-STREET', 'I-STREET']


# Представление адреса в виде слов-токенов и тегов NER

In [65]:
str ='Алтайский Край, Залесовский Район, Залесово Село, автодорога Мартыново-Тогул-Залесово Территория, 151-й Километр'
tokens = str.split(' ')
tokens

['Алтайский',
 'Край,',
 'Залесовский',
 'Район,',
 'Залесово',
 'Село,',
 'автодорога',
 'Мартыново-Тогул-Залесово',
 'Территория,',
 '151-й',
 'Километр']

### Функция конвертации набора адресо-образующих элементов в токены и NER-теги

In [66]:
def ner_tags_for_type(type : ElementType , n: int):
    if n==0:
        return []
    else:
        return [type.begin_tag_code()] + [type.inside_tag_code()] * (n - 1)

In [67]:
ner_tags_for_type(ElementType.REGION, 3)

[0, 1, 1]

In [68]:
def convert_address_elements(elements: dict ) :
    region_tokens = elements['region_name'].split(' ') + elements['region_type_name_short'].split(' ')
    area_tokens = elements['area_name'].split(' ')+elements['area_type_name_short'].split(' ')
    city_tokens = elements['city_name'].split(' ')+elements['city_type_name_short'].split(' ')
    territory_tokens = elements['territory_name'].split(' ')+elements['territory_type_name_short'].split(' ')
    street_tokens = elements['street_name'].split(' ')+elements['street_type_name_short'].split(' ')

    region_tokens = list(filter(('').__ne__, region_tokens))
    area_tokens = list(filter(('').__ne__, area_tokens))
    city_tokens= list(filter(('').__ne__, city_tokens))
    territory_tokens = list(filter(('').__ne__, territory_tokens))
    street_tokens = list(filter(('').__ne__, street_tokens))
    
    region_ner_tags = ner_tags_for_type(ElementType.REGION, len(region_tokens))
    area_ner_tags = ner_tags_for_type(ElementType.AREA, len(area_tokens))
    city_ner_tags = ner_tags_for_type(ElementType.CITY, len(city_tokens))
    territory_ner_tags = ner_tags_for_type(ElementType.TERRITORY, len(territory_tokens))
    street_ner_tags = ner_tags_for_type(ElementType.STREET, len(street_tokens))
    
    tokens = region_tokens+area_tokens+city_tokens+territory_tokens+street_tokens
    ner_tags = region_ner_tags+area_ner_tags+city_ner_tags+territory_ner_tags+street_ner_tags
    
    return {'tokens': tokens, 'ner_tags': ner_tags}
    # tokens = tokens.append


In [69]:
def convert_address_elements_df(df:pd.DataFrame):
    return [convert_address_elements(item) for item in df.to_dict('records')]

convert_address_elements_df(r)


[{'tokens': ['Алтайский',
   'край',
   'Залесовский',
   'р-н',
   'Залесово',
   'с',
   'Полевая',
   'ул'],
  'ner_tags': [0, 1, 2, 3, 6, 7, 8, 9]}]

## Appply to dataframe

In [70]:
s = df.apply(convert_address_elements, axis=1)

In [71]:
new_df = pd.DataFrame(list(s))

In [72]:
new_df

,tokens,ner_tags
0,"[Алтайский, край, Залесовский, р-н, Залесово, ...","[0, 1, 2, 3, 6, 7, 8, 9]"
1,"[Алтайский, край, Залесовский, р-н, Залесово, ...","[0, 1, 2, 3, 6, 7, 4, 5, 5, 8, 9]"
2,"[Алтайский, край, Залесовский, р-н, Залесово, ...","[0, 1, 2, 3, 6, 7, 8, 9]"
3,"[Алтайский, край, Залесовский, р-н, Залесово, ...","[0, 1, 2, 3, 6, 7, 8, 9]"
4,"[Алтайский, край, Залесовский, р-н, Залесово, ...","[0, 1, 2, 3, 6, 7, 8, 9]"
...,...,...
612825,"[Еврейская, Аобл, Биробиджан, г, СОТ, Биршоссе...","[0, 1, 6, 7, 4, 5, 5, 5, 5, 8, 9]"
612826,"[Еврейская, Аобл, Биробиджан, г, СОТ, Биршоссе...","[0, 1, 6, 7, 4, 5, 5, 5, 5, 8, 9]"
612827,"[Еврейская, Аобл, Биробиджан, г, СОТ, Биршоссе...","[0, 1, 6, 7, 4, 5, 5, 5, 5, 8, 9]"
612828,"[Еврейская, Аобл, Биробиджан, г, СОТ, Багульни...","[0, 1, 6, 7, 4, 5, 5, 8, 9]"


## В формате Datasets

In [73]:
dataset = Dataset.from_pandas(new_df)

In [74]:
train_testvalid = dataset.train_test_split(test_size=0.33, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']})

In [75]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 410596
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 101117
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 101117
    })
})

In [76]:
dataset.save_to_disk('address_elements')

Saving the dataset (0/1 shards):   0%|          | 0/410596 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/101117 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/101117 [00:00<?, ? examples/s]